In [ ]:
import Pkg;
Pkg.instantiate()
using DrugResponseModel

# import data from the path
conc_l, pop, g2, g1, g2_0, g1_0 = setup_data("lapatinib"); # in which:

lowEC50 = 50.0
highEC50 = 350.0
costt, ps = optimize_hill(lowEC50, highEC50, conc_l, g1, g2, g1_0, g2_0)

In [ ]:
costt

In [ ]:
nG1, nG2, effects = getODEparams(ps, conc_l)

In [ ]:
ODEplot_all(effects, g1, g2, g1_0, g2_0, pop, nG1, nG2)